In [ ]:
import skimage
from skimage import io
import matplotlib.pyplot as plt
import json
import cv2
import numpy as np
import matplotlib
import os

In [ ]:
def integrate_parallel(dataset = 'train'):

    mask = skimage.io.imread('data_WiSAR\\data\\mask.png')
    if not os.path.isdir('data_WiSAR\\data\\integrated_imgs_' + dataset):
        os.makedirs('data_WiSAR\\data\\integrated_imgs_' + dataset)
    for folder in os.listdir('data_WiSAR\\data\\' + dataset):
        if folder.startswith(dataset):
            if not os.path.isdir('data_WiSAR\\data\\integrated_imgs_' + dataset + '\\' + folder):
                os.makedirs('data_WiSAR\\data\\integrated_imgs_' + dataset + '\\' + folder)
            with open('data_WiSAR\\data\\' + dataset + '\\' + folder + '\\homographies.json') as f:
                homographies = json.load(f)
            for time_point in [0,1,2,3,4,5,6]:
                warped_imgs = []
                for camera in ['B01', 'B02', 'B03', 'B04', 'B05', 'G01', 'G02', 'G03', 'G04', 'G05']:
                    img = skimage.io.imread('data_WiSAR\\data\\' + dataset + '\\' + folder + '\\' + str(time_point) + '-' + camera + '.png')
                    img = img[..., :3].astype(np.float32)/255
                    img[mask==0] = np.nan

                    homography = np.array(homographies[str(time_point) + '-' + camera])
                    warped_img = img.copy()
                    for dim in range(3):
                        warped_img[:,:,dim] = cv2.warpPerspective(img[:,:,dim],homography,img.shape[:2],borderMode = cv2.BORDER_CONSTANT,borderValue = np.nan) 
                    warped_imgs.append(warped_img)

                integrated_img = np.nanmean(np.array(warped_imgs), axis = 0)

                integrated_img_nan = integrated_img.copy()
                for dim in range(3):
                    integrated_img_nan[:,:,dim] = np.where(np.isnan(warped_imgs).any(axis = 0).all(axis = 2), np.nan, integrated_img[:,:,dim])

            #     plt.figure()
            #     plt.imshow(integrated_img)
            #     plt.figure()
            #     plt.imshow(integrated_img_nan)
            
                matplotlib.image.imsave('data_WiSAR\\data\\integrated_imgs_' + dataset + '\\' + folder + '\\' + str(time_point) + '-' + camera + '_integrated.png', integrated_img)
                matplotlib.image.imsave('data_WiSAR\\data\\integrated_imgs_' + dataset + '\\' + folder + '\\' + str(time_point) + '-' + camera + '_integrated_nan.png', integrated_img_nan)


In [ ]:
def integrate_all(dataset = 'train'):

    mask = skimage.io.imread('data_WiSAR\\data\\mask.png')
    if not os.path.isdir('data_WiSAR\\data\\integrated_imgs_' + dataset):
        os.makedirs('data_WiSAR\\data\\integrated_imgs_' + dataset)
    for folder in os.listdir('data_WiSAR\\data\\' + dataset):
        if folder.startswith(dataset):
            if not os.path.isdir('data_WiSAR\\data\\integrated_imgs_' + dataset + '\\' + folder):
                os.makedirs('data_WiSAR\\data\\integrated_imgs_' + dataset + '\\' + folder)
            with open('data_WiSAR\\data\\' + dataset + '\\' + folder + '\\homographies.json') as f:
                homographies = json.load(f)
            warped_imgs = []
            for time_point in [0,1,2,3,4,5,6]:
                for camera in ['B01', 'B02', 'B03', 'B04', 'B05', 'G01', 'G02', 'G03', 'G04', 'G05']:
                    img = skimage.io.imread('data_WiSAR\\data\\' + dataset + '\\' + folder + '\\' + str(time_point) + '-' + camera + '.png')
                    img = img[..., :3].astype(np.float32)/255
                    img[mask==0] = np.nan

                    homography = np.array(homographies[str(time_point) + '-' + camera])
                    warped_img = img.copy()
                    for dim in range(3):
                        warped_img[:,:,dim] = cv2.warpPerspective(img[:,:,dim],homography,img.shape[:2],borderMode = cv2.BORDER_CONSTANT,borderValue = np.nan) 
                    warped_imgs.append(warped_img)

            integrated_img = np.nanmean(np.array(warped_imgs), axis = 0)

            integrated_img_nan = integrated_img.copy()
            for dim in range(3):
                integrated_img_nan[:,:,dim] = np.where(np.isnan(warped_imgs).any(axis = 0).all(axis = 2), np.nan, integrated_img[:,:,dim])

    #         plt.figure()
    #         plt.imshow(integrated_img)
    #         plt.figure()
    #         plt.imshow(integrated_img_nan)
    
            matplotlib.image.imsave('data_WiSAR\\data\\integrated_imgs_' + dataset + '\\' + folder + '\\all_integrated.png', integrated_img)
            matplotlib.image.imsave('data_WiSAR\\data\\integrated_imgs_' + dataset + '\\' + folder + '\\all_integrated_nan.png', integrated_img_nan)


In [ ]:
integrate_parallel('train')
integrate_parallel('test')
integrate_parallel('validation')

integrate_all('train')
integrate_all('test')
integrate_all('validation')